In [30]:
import pandas as pd

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### We read in the price data, the # of shareholder data vix data and market cap data and sp500 data


We first read in the `sp500.csv`

In [32]:
symbols = pd.read_csv('/content/drive/My Drive/edav/SP500 Components.csv', index_col=False)
symbols = symbols[['Symbol']]

In [33]:
#symbols = pd.read_csv('/content/drive/My Drive/edav/sp500.csv', index_col=False)
symbols = list(symbols['Symbol'].values)
symbols = sorted(symbols)

In [34]:
prices = pd.read_csv('/content/drive/My Drive/edav/data/sp500_data.csv').set_index(['Date', 'Symbol'])
holders = pd.read_csv('/content/drive/My Drive/edav/data/sp500_users_holding.csv').set_index(['Date', 'Symbol'])
vix = pd.read_csv("/content/drive/My Drive/edav/VIX.csv")

We make sure that Data is of datetimetype

In [35]:
def fix_date(df):
  df = df.reset_index()
  df['Date'] =pd.to_datetime(df['Date']).dt.date
  return df.set_index(['Date', 'Symbol'])

holders = fix_date(holders)
prices = fix_date(prices)

In [36]:
print(holders.shape)
print(prices.shape)

(350365, 1)
(256253, 3)


read in market cap and do some shifting around to bring it to the correct format

In [37]:
def rename_stocks(df):
  df.replace("BF/B","BF.B",inplace=True)
  df.replace("BRK/B","BRK.B",inplace=True)
  return df

m_cap = pd.read_excel("/content/drive/MyDrive/edav/SP500 Market Cap.xlsx",)
s = list(m_cap.columns)[1:]

# fix date column
m_cap = m_cap.rename(columns={'Unnamed: 0':'Date'})
m_cap['Date'] = m_cap['Date'].astype(str)
m_cap['Date'] = pd.to_datetime(m_cap['Date']).dt.date
m_cap = m_cap.set_index(['Date'])

# pivotlong
m_cap = pd.DataFrame(m_cap.stack(), columns=["Mcap"])

# Cleaning up
m_cap = m_cap.reset_index().rename(columns={'level_1':'Symbol'})
m_cap['Symbol'] = m_cap['Symbol'].astype(str)

#Keep only sp500
m_cap = m_cap[m_cap['Symbol'].isin(symbols)]

m_cap = rename_stocks(m_cap)
m_cap = m_cap.set_index(['Date', 'Symbol'])

### We merge them together with key "Date" and "Symbol". We expect that for each date we will have ~500 "Symbols"

In [38]:
result = pd.merge(prices, holders,left_index=True, right_index=True, how ='outer',sort=True)

In [39]:
result = result.drop(columns=['Unnamed: 0'])

In [40]:
result = result.reset_index()


Bring vix into correct format to merge

In [41]:
vix['Date'] = pd.to_datetime(vix['Date']).dt.date

vix_adj_close = list(vix['Adj Close'])
vix_date = list(vix['Date'])

dict_vix = {}
for i in range(0,len(vix_date)):
  dict_vix[vix_date[i]] = vix_adj_close[i]

result['vix_Adj_close'] = result['Date'].map(dict_vix)

In [42]:
result = result.set_index(['Date', 'Symbol'])

merge now with Market cap

In [43]:
result = pd.merge(result, m_cap,left_index=True, right_index=True, how ='outer',sort=True)

In [44]:
result

Adj Close       Volume  ...  vix_Adj_close         Mcap
Date       Symbol                           ...                            
2018-08-01 A        63.723564    2679200.0  ...          13.15   20790.4891
           AAL      37.332352    7080700.0  ...          13.15   17563.8215
           AAP     138.336090     540200.0  ...          13.15   10345.8757
           AAPL     48.465130  271742800.0  ...          13.15  973230.0890
           ABBV     82.121834    5330400.0  ...          13.15  142185.7424
...                       ...          ...  ...            ...          ...
2020-09-01 YUM            NaN          NaN  ...            NaN   28979.5743
           ZBH            NaN          NaN  ...            NaN   29086.3598
           ZBRA           NaN          NaN  ...            NaN   15317.0696
           ZION           NaN          NaN  ...            NaN    5277.7192
           ZTS            NaN          NaN  ...            NaN   76997.1993

[377382 rows x 5 columns]

# Sanity check

We now should make sure that in the resulting dataframe there is a row for each trading date between **August 2018** and **August 2020**. 

We should also make sure that for each date there are 500 different Symbols for which we record 

`Adj Close`	`Volume`	 `users_holding` 


## Sanity Check about the dates

In [45]:
!pip install pandas_market_calendars

The following library returns all trading days between the two dates we are interested in

In [46]:
import pandas_market_calendars as mcal

nyse = mcal.get_calendar('NYSE')
early = nyse.schedule(start_date='2018-08-1', end_date='2020-08-1').reset_index()
early['index'] = early['index'].dt.date

trading_days = list(early['index'])
trading_days = [str(i) for i in trading_days]

We find all unique dates in our resulting dataframe

In [47]:
result_dates = pd.unique(result.reset_index()['Date'])

In [48]:
print('Our dates {}'.format(len(result_dates)))
print('Trading dates {}'.format(len(trading_days)))

Our dates 749
Trading dates 504


Ideally we want the to lengths to match. I think Robintrack includes weekends aswell. I think we should remove all non-trading dates.

## Sanity Check about the tickers

In [49]:
len(symbols)

505

# To find the companies that we are missing, I am only looking at trading days

Note:  The following piece of code

```
result.reset_index().groupby('Date')
```
will group by the unindexed dataframe by Date and for each group it will calculate if its length is smaller than ```len(symbols)```



In [50]:
set_bad_Stocks = {}
for name, group in result.reset_index().groupby('Date'):
  if name not in trading_days:
    continue
  # if there are more tickers that we find in each group
  if len(symbols) - len(group) > 0:
    for item in symbols:
      if item not in list(group['Symbol']):
        if item in set_bad_Stocks.keys():
          set_bad_Stocks[item] = set_bad_Stocks[item] + 1
        else:
          set_bad_Stocks[item] = 1
  if len(symbols) - len(group) > 0:
    print("No data for date: {}, Number of missing tickers: {}".format(name, len(symbols) - len(group)))


The following symbols are consistently missing(No entry for them at all)

In [51]:
set_bad_Stocks

{}

## I propose we drop all non trading days and we also drop these problematic companies for which sometimes we don't have an entry at all or enter them with Nans everywhere

# Briefly looking at Nans

Assuming we dropped the non trading days, which rows have NaN?

In [52]:
df = result.reset_index()
df = df[df['Date'].isin(trading_days)]
is_NaN = df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df[row_has_NaN]

print(rows_with_NaN)

             Date Symbol   Adj Close  ...  users_holding  vix_Adj_close        Mcap
13     2018-08-01    ADS  216.055618  ...     166.500000      13.150000         NaN
20     2018-08-01    AIV   28.400883  ...            NaN      13.150000   6729.8663
31     2018-08-01   AMCR    9.932543  ...            NaN      13.150000         NaN
62     2018-08-01   BF.B   51.215580  ...            NaN      13.150000         NaN
64     2018-08-01    BIO  311.730011  ...            NaN      13.150000   9286.8119
...           ...    ...         ...  ...            ...            ...         ...
365654 2020-07-31    NBL    9.970695  ...   18246.791667      24.459999         NaN
365705 2020-07-31   POOL  315.550201  ...            NaN      24.459999  12692.0293
365760 2020-07-31    TDY  306.700012  ...            NaN      24.459999  11306.0106
365762 2020-07-31    TER   88.855766  ...            NaN      24.459999  14746.7473
365780 2020-07-31    TYL  357.250000  ...            NaN      24.459999  143

Which columns have Nan

In [53]:
nan_values = df.isna()
nan_columns = nan_values.any()

columns_with_nan = df.columns[nan_columns].tolist()
print(columns_with_nan)

['Adj Close', 'Volume', 'users_holding', 'vix_Adj_close', 'Mcap']


In [59]:
df[df[['Volume']].isna().any(axis=1)]

,Date,Symbol,Adj Close,Volume,users_holding,vix_Adj_close,Mcap
289,2018-08-01,LUMN,NaN,NaN,NaN,NaN,20034.1766
469,2018-08-01,VIAC,NaN,NaN,NaN,NaN,19933.6390
796,2018-08-02,LUMN,NaN,NaN,NaN,NaN,20206.7921
976,2018-08-02,VIAC,NaN,NaN,NaN,NaN,19834.9355
1303,2018-08-03,LUMN,NaN,NaN,NaN,NaN,20314.6767
...,...,...,...,...,...,...,...
363563,2020-07-27,LUMN,NaN,NaN,NaN,NaN,10645.7706
364076,2020-07-28,LUMN,NaN,NaN,NaN,NaN,10854.2960
364589,2020-07-29,LUMN,NaN,NaN,NaN,NaN,10799.4209
365102,2020-07-30,LUMN,NaN,NaN,NaN,NaN,10689.6707


In [60]:
df[df[['Volume']].isna().any(axis=1)]

,Date,Symbol,Adj Close,Volume,users_holding,vix_Adj_close,Mcap
289,2018-08-01,LUMN,NaN,NaN,NaN,NaN,20034.1766
469,2018-08-01,VIAC,NaN,NaN,NaN,NaN,19933.6390
796,2018-08-02,LUMN,NaN,NaN,NaN,NaN,20206.7921
976,2018-08-02,VIAC,NaN,NaN,NaN,NaN,19834.9355
1303,2018-08-03,LUMN,NaN,NaN,NaN,NaN,20314.6767
...,...,...,...,...,...,...,...
363563,2020-07-27,LUMN,NaN,NaN,NaN,NaN,10645.7706
364076,2020-07-28,LUMN,NaN,NaN,NaN,NaN,10854.2960
364589,2020-07-29,LUMN,NaN,NaN,NaN,NaN,10799.4209
365102,2020-07-30,LUMN,NaN,NaN,NaN,NaN,10689.6707


# Drop not non trading days and save only August 2018-2020

In [61]:
df

,Date,Symbol,Adj Close,Volume,users_holding,vix_Adj_close,Mcap
0,2018-08-01,A,63.723564,2679200.0,712.208333,13.150000,20790.4891
1,2018-08-01,AAL,37.332352,7080700.0,4937.916667,13.150000,17563.8215
2,2018-08-01,AAP,138.336090,540200.0,283.958333,13.150000,10345.8757
3,2018-08-01,AAPL,48.465130,271742800.0,154792.750000,13.150000,973230.0890
4,2018-08-01,ABBV,82.121834,5330400.0,14192.750000,13.150000,142185.7424
...,...,...,...,...,...,...,...
365829,2020-07-31,YUM,90.606827,1914900.0,6601.333333,24.459999,27406.0500
365830,2020-07-31,ZBH,134.622543,1309300.0,1023.750000,24.459999,27889.3330
365831,2020-07-31,ZBRA,280.750000,472800.0,1076.333333,24.459999,14975.1612
365832,2020-07-31,ZION,31.881811,1442500.0,717.916667,24.459999,5324.3657


In [62]:
df = result.reset_index()
df = df[df['Date'].isin(trading_days)]


In [58]:
df.to_csv("/content/drive/My Drive/edav/final_df.csv",index=False)